<a href="https://colab.research.google.com/github/ml2-picme/PicMe/blob/master/Model%20Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the PyDrive wrapper & import libraries
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
from keras.applications.vgg16 import VGG16, preprocess_input

from PIL import Image
import numpy as np

from keras.applications.vgg16 import decode_predictions
import matplotlib.pyplot as plt

#import plotly.plotly as plotly

from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go

# Define the Methods

localFilePath = "/root/data"

def reloadPictures():
  # Authenticate and create the PyDrive client
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Delete the local directory
  !rm -r $localFilePath
  print("Deleted local directory")

  # Set a local directory and upload files from Google Drive
  local_root_path = os.path.expanduser(localFilePath)
  try:
    os.makedirs(local_root_path)
  except: pass

  def ListFolder(google_drive_id, destination):
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % google_drive_id}).GetList()
    counter = 0
    for f in file_list:
      # If it is a directory then, create the dicrectory and upload the file inside it
      if f['mimeType']=='application/vnd.google-apps.folder': 
        folder_path = os.path.join(destination, f['title'])
        os.makedirs(folder_path)
        print('* creating directory {}'.format(folder_path))
        ListFolder(f['id'], folder_path)
      else:
        fname = os.path.join(destination, f['title'])
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)
        counter += 1
    print('* {} files were uploaded in {}'.format(counter, destination))

  # Opening and uploading the Drive Folder => this is the shared directory and should work for everyone who belongs to the project
  print("\nCalling the ListFolder method, that uploads the data...")
  ListFolder("1M7TDDTFoNOjsrlEMonWN858uQONUp9Pz", local_root_path)
  print("Upload finished!")

  # Show the Data in the local Path
  print("\nListing files in local test directory")
  !ls -l $localFilePath
  
  
def createFileList():
  files = []
  # r=root, d=directories, f = files
  for r, d, f in os.walk(localFilePath):
      for file in f:
          if file.endswith(".jpg"):
              files.append(os.path.join(r, file))

  print("Printing files:")

  for f in files:
      print(f)
      
  return files

def prepareImagesForClassification(files):
  
  images = []
  
  for file in files:
    try:
      image = Image.open(file)
      
      image = image.resize((224, 224), Image.LANCZOS)
      
      image = image.convert("RGB")
      
      image = np.asarray(image)
      
      images.append(image)
    except OSError:
      pass
  
  images = np.asarray(images)
  
  return images

def classifyImages(images, model):
  
  # preprocess the images to fit to the model
  images_preprocessed = preprocess_input(images)
  
  # use the model to classifi the images
  images_pred = model.predict(images_preprocessed, verbose=1)
  
  pred_results = decode_predictions(images_pred)
  
  return pred_results

def printResults(images, predictedClasses):
  
  for i in range(len(images)):
    plt.figure()
    plt.imshow(images[i])
    plt.title(predictedClasses[i])
    plt.axis('off')
    
  plt.show()
  
def compareResults(fileNames, resulsList, modelList):
  
  for i in range(len(fileNames)):
    print("Comparing the Results for File: " + fileNames[i])
    
    for j in range(5):
      
      for k in range(len(modelList)):
        
        resultToPrint = resultsList[k][i][j]
        
        # Setze 5% als Threshold
        if(resultToPrint is not None and resultToPrint[2] > 0.05):
          print(" > ", (j+1), ". Platz @ ", modelList[k], " : ", resultsList[k][i][j])
        else:
          print(" > ", (j+1), ". Platz @ ", modelList[k], " : ", "---")
      
      print("=================================================================")
      
    print("")

In [0]:
# Lade die Bilder erneut
reloadPictures()
foundFiles = createFileList()
preparedImages = prepareImagesForClassification(foundFiles)




In [95]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19

# VGG 16
predictedClassesVGG16 = classifyImages(preparedImages, VGG16(input_shape=(224, 224, 3)))
printResults(preparedImages, predictedClassesVGG16)

# VGG 19
predictedClassesVGG19 = classifyImages(preparedImages, VGG19(input_shape=(224, 224, 3)))
printResults(preparedImages, predictedClassesVGG19)

resultsList = [predictedClassesVGG16, predictedClassesVGG19]

modelList = ['VGG16', 'VGG19']

# Diese Methode hier braucht 3 Parameter:
# 1. Die Liste der Dateinamen
# 2. Die Ergebnisse der einzelnen Modellen, als Array zusammengefasst
# 3. Die Namen der Modelle, einfach als String Array
compareResults(foundFiles, resultsList, modelList)



Comparing the Results for File: /root/data/handy.jpg
 >  1 . Platz @  VGG16  :  ('n03584254', 'iPod', 0.3981493)
 >  1 . Platz @  VGG19  :  ('n03584254', 'iPod', 0.39623585)
 >  2 . Platz @  VGG16  :  ---
 >  2 . Platz @  VGG19  :  ('n02992529', 'cellular_telephone', 0.20556368)
 >  3 . Platz @  VGG16  :  ---
 >  3 . Platz @  VGG19  :  ---
 >  4 . Platz @  VGG16  :  ---
 >  4 . Platz @  VGG19  :  ---
 >  5 . Platz @  VGG16  :  ---
 >  5 . Platz @  VGG19  :  ---

Comparing the Results for File: /root/data/brücke.jpg
 >  1 . Platz @  VGG16  :  ('n04311004', 'steel_arch_bridge', 0.95547444)
 >  1 . Platz @  VGG19  :  ('n04311004', 'steel_arch_bridge', 0.8680669)
 >  2 . Platz @  VGG16  :  ---
 >  2 . Platz @  VGG19  :  ---
 >  3 . Platz @  VGG16  :  ---
 >  3 . Platz @  VGG19  :  ---
 >  4 . Platz @  VGG16  :  ---
 >  4 . Platz @  VGG19  :  ---
 >  5 . Platz @  VGG16  :  ---
 >  5 . Platz @  VGG19  :  ---

Comparing the Results for File: /root/data/hund.jpg
 >  1 . Platz @  VGG16  :  ('n0